### Installation and Setup

First, we need to intall the package

In [4]:
] add PyCall

  Updating registry at `C:\Users\Arnav Sood\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `C:\Users\Arnav Sood\.julia\environments\v1.0\Project.toml`
 [no changes]
  Updating `C:\Users\Arnav Sood\.julia\environments\v1.0\Manifest.toml`
 [no changes]


In [5]:
] add Conda

 Resolving package versions...
  Updating `C:\Users\Arnav Sood\.julia\environments\v1.0\Project.toml`
 [no changes]
  Updating `C:\Users\Arnav Sood\.julia\environments\v1.0\Manifest.toml`
 [no changes]


This installs a Conda installation in the user depot, which we can see as follows

In [6]:
using PyCall, Conda 
Conda.ROOTENV 
Conda.PYTHONDIR 

"C:\\Users\\Arnav Sood\\.julia\\conda\\3"

Next, we need to install the actual solver package

In [7]:
Conda.add("pip") # Adds the pip installer

┌ Info: Running `conda install -y pip` in root environment
└ @ Conda C:\Users\Arnav Sood\.julia\packages\Conda\uQitS\src\Conda.jl:112


Solving environment: ...working... done

# All requested packages already installed.



For now, we need to use the actual Python executable in the path above to install the solver.

In [ ]:
from pip._internal import main 
main(['install', '--pre', 'dfogn'])

We can see the package from Julia now

In [8]:
@pyimport dfogn

### Usage

Here's a simple, vanilla example

In [9]:
# Rosenbrock
rosenbrock(x) = [10. *(x[2]-x[1])^2, 1. - x[1]]
x0 = [-1.2, 1.]
soln = dfogn.solve(rosenbrock, x0)

PyObject <dfogn.dfogn.OptimResults object at 0x00000000011CB588>

In [11]:
# Unpack the results
@show soln[:x] # optimum
@show soln[:resid] # residual at optimum
@show soln[:f] # value of objective at optimum 
@show soln[:msg] # halting reason 
@show soln[:jacobian] # empirical jacobian 

# etc. 

# The key is to unpack with symbols soln[:fieldname] as opposed to soln.fieldname

soln[:x] = [1.0, 1.0002]
soln[:resid] = [3.98096e-7, 0.0]
soln[:f] = 1.5848034648780586e-13
soln[:msg] = "Success: Objective is sufficiently small"
soln[:jacobian] = [-0.00836832 0.00836814; -1.0 -4.86754e-14]


2×2 Array{Float64,2}:
 -0.00836832   0.00836814 
 -1.0         -4.86754e-14

Now, let's try this with a more relevant example.

In [12]:
# Setup and solve the version with callbacks to Julia objects.
vals = []
function callbackprob(x)
    push!(vals, x) # analogous to pushing to a dataframe, etc.
    return [10. *(x[2]-x[1])^2, 1. - x[1]]
end
dfogn.solve(callbackprob, x0)

# Show callbacks.
@show vals

vals = Any[[-1.2, 1.0], [-1.08, 1.0], [-1.2, 1.12], [-0.997443, 0.912912], [-0.682055, 0.551069], [1.0, 1.14167], [1.07289, 0.934084], [1.0, 1.21455], [1.0, 1.08533], [1.0, 1.06105], [1.0, 0.966903], [1.0, 0.86672], [0.999508, 0.972883], [0.997946, 0.978676], [1.0, 0.983149], [1.0, 0.988834], [1.0, 1.01358], [1.00243, 0.988343], [0.999779, 0.991642], [1.0, 0.994337], [1.0, 1.00122], [1.00223, 1.00164], [0.999757, 1.00251], [1.0, 1.00089], [1.0, 1.00069], [1.0, 1.00039], [1.0, 0.999434], [0.99988, 1.00039], [1.0, 1.00051], [1.0, 1.00033], [1.0, 1.0002]]


31-element Array{Any,1}:
 [-1.2, 1.0]          
 [-1.08, 1.0]         
 [-1.2, 1.12]         
 [-0.997443, 0.912912]
 [-0.682055, 0.551069]
 [1.0, 1.14167]       
 [1.07289, 0.934084]  
 [1.0, 1.21455]       
 [1.0, 1.08533]       
 [1.0, 1.06105]       
 [1.0, 0.966903]      
 [1.0, 0.86672]       
 [0.999508, 0.972883] 
 ⋮                    
 [1.0, 0.994337]      
 [1.0, 1.00122]       
 [1.00223, 1.00164]   
 [0.999757, 1.00251]  
 [1.0, 1.00089]       
 [1.0, 1.00069]       
 [1.0, 1.00039]       
 [1.0, 0.999434]      
 [0.99988, 1.00039]   
 [1.0, 1.00051]       
 [1.0, 1.00033]       
 [1.0, 1.0002]        

There is also functionality in DFOGN to impose bounds